In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as soup

# Data Preparation

## "Best Value"

## 0 - 30 hotels

In [174]:
html_main = requests.get("https://www.tripadvisor.com/Hotels-g32655-Los_Angeles_California-Hotels.html")
bsobj_main = soup(html_main.content, "lxml")

In [175]:
rank = []
hotel_name = []
rate_score = []
hotel_link = []
for s1 in bsobj_main.find_all("div", class_ = "ui_column is-8 main_col allowEllipsis"):
    if s1.find("span", class_ = "ui_merchandising_pill sponsored_v2") == None:
        for s2 in s1.find_all("div", class_ = "listing_title"):
            hotel_name.append(s1.a.text[6:])
            hotel_link.append(s1.a["href"])
        for s3 in s1.find_all("div", class_ = "info-col"):
            rate_score.append(s3.a["alt"])
        for s4 in s1.find_all("ul", class_ = "icons_list easyClear vertical"):
            for s5 in s4.find_all("div", class_ = "label"):
                label.append(s5.text)

for s6 in bsobj_main.find_all("div", class_ = "popindex"):    
    rank.append(s6.text.split(" ")[0][1:])

## 31 - 150 hotels

In [176]:
top_150 = list(range(30, 150, 30))

In [177]:
for num in top_150:
    html_main = requests.get("https://www.tripadvisor.com/Hotels-g32655-oa" + str(num) + "-Los_Angeles_California-Hotels.html")
    bsobj_main = soup(html_main.content, "lxml")
    
    for s1 in bsobj_main.find_all("div", class_ = "ui_column is-8 main_col allowEllipsis"):
        if s1.find("span", class_ = "ui_merchandising_pill sponsored_v2") == None:
            for s2 in s1.find_all("div", class_ = "listing_title"):
                hotel_name.append(s1.a.text[6:])
                hotel_link.append(s1.a["href"])
            for s3 in s1.find_all("div", class_ = "info-col"):
                rate_score.append(s3.a["alt"])
            for s4 in s1.find_all("ul", class_ = "icons_list easyClear vertical"):
                for s5 in s4.find_all("div", class_ = "label"):
                    label.append(s5.text)

    for s6 in bsobj_main.find_all("div", class_ = "popindex"):    
        rank.append(s6.text.split(" ")[0][1:])

In [178]:
hotel_link_update = []
for site in hotel_link:
    hotel_link_update.append("https://www.tripadvisor.com/" + site)

In [180]:
data = {"Rank": rank, "Hotel Name": hotel_name, "Rate": rate_score, "Site Link": hotel_link_update}
df = pd.DataFrame.from_dict(data)
df

,Rank,Hotel Name,Rate,Site Link
0,1,The Hollywood Roosevelt,4.5 of 5 bubbles,https://www.tripadvisor.com//Hotel_Review-g326...
1,2,Hotel Figueroa,4.5 of 5 bubbles,https://www.tripadvisor.com//Hotel_Review-g326...
2,3,Hilton Los Angeles/Universal City,4 of 5 bubbles,https://www.tripadvisor.com//Hotel_Review-g326...
3,4,Hotel Erwin,4.5 of 5 bubbles,https://www.tripadvisor.com//Hotel_Review-g326...
4,5,Hollywood Hotel,4 of 5 bubbles,https://www.tripadvisor.com//Hotel_Review-g326...
...,...,...,...,...
145,146,Radisson Hotel Chatsworth,4 of 5 bubbles,https://www.tripadvisor.com//Hotel_Review-g326...
146,147,Regency Inn Los Angeles,4 of 5 bubbles,https://www.tripadvisor.com//Hotel_Review-g326...
147,148,Samesun Hollywood,4 of 5 bubbles,https://www.tripadvisor.com//Hotel_Review-g326...
148,149,The NoMad Hotel Los Angeles,4.5 of 5 bubbles,https://www.tripadvisor.com//Hotel_Review-g326...


# Location After Input

In [181]:
hotel_name

['The Hollywood Roosevelt',
 'Hotel Figueroa',
 'Hilton Los Angeles/Universal City',
 'Hotel Erwin',
 'Hollywood Hotel',
 'Hotel June',
 'Loews Hollywood Hotel',
 'Hotel Indigo Los Angeles Downtown',
 'Sheraton Grand Los Angeles',
 'InterContinental Los Angeles Downtown',
 'Sheraton Gateway Los Angeles Hotel',
 'La Quinta Inn & Suites by Wyndham Lax',
 'Hotel Angeleno',
 'JW Marriott Los Angeles L.A. LIVE',
 'Blvd Hotel & Spa',
 'The Westin Bonaventure Hotel & Suites, Los Angeles',
 'H Hotel Los Angeles, Curio Collection by Hilton',
 'SLS Hotel, a Luxury Collection Hotel, Beverly Hills',
 'Hilton Los Angeles Airport',
 'The Westin Los Angeles Airport',
 'Venice Suites',
 'Millennium Biltmore Los Angeles',
 'Embassy Suites by Hilton LAX North',
 'Renaissance Los Angeles Airport Hotel',
 'Hilton Garden Inn Los Angeles Hollywood',
 'Su Casa Venice Beach',
 'W Hollywood',
 'Homewood Suites by Hilton Los Angeles International Airport',
 'The LINE Hotel Los Angeles',
 'Luxe Sunset Boulevard 

In [182]:
def visit_hotel(df, want_to_go_name):
    each_name = want_to_go_name.split(".")
    
    for name in each_name:
        if name == each_name[0]:
            visit_html = df[["Hotel Name", "Site Link"]][df["Hotel Name"] == name]
        else:
            visit_html = visit_html.append(df[["Hotel Name", "Site Link"]][df["Hotel Name"] == name])
    
    return visit_html

In [183]:
want_to_go_name = str(input("Which hotel do you want to live?"))
visit_html = visit_hotel(df, want_to_go_name)

Which hotel do you want to live?Los Angeles Airport Marriott


In [184]:
def find_location(visit_html):
    
    location = []
    
    for link in visit_html["Site Link"]:
        html = requests.get(link)
        bsobj = soup(html.content, "lxml")
    
        for loc in bsobj.find_all("script", type = "application/ld+json"):
            if "streetAddress" in loc.string:
                find_part_html = loc.string.split("{")
                for street in find_part_html:
                    if "streetAddress" in street:
                        for value in street.split(","):
                            if "streetAddress" in value:
                                street_name = value.split(":")[1][1:][:-1] + ", "
                                   
                            if "addressLocality" in value:
                                city_name = value.split(":")[1][1:][:-1] + ", "
                                 
                            if "postalCode" in value:
                                zipcode = "CA " + value.split(":")[1][1:][:-1]
                            
        location_info = street_name + city_name + zipcode
        location.append(location_info)
    
    return location

In [185]:
location = find_location(visit_html)

In [186]:
location

['5855 West Century Boulevard, Los Angeles, CA 90045']